In [1]:
from flatfindr.facebook import Facebook
from flatfindr.utils import KEYWORDS
from collections import defaultdict
from time import sleep
import random
from PIL import Image
import requests
from io import BytesIO
%load_ext autoreload
%autoreload 2

In [2]:
fb = Facebook()
fb.log_in()

In [7]:
fb = Facebook()
fb.driver.get('file:///Users/aduverger/code/aduverger/flatfindr/raw_data/test2')

In [7]:
index = random.randint(0, len(item_links) - 1)
item_url = item_links[index]
db = {}
db["url"] = item_url

fb.driver.get(item_url)
# Click on 'see more' button to have the full description
buttons = fb.driver.find_elements_by_xpath("//div[@role='button']")
_ = [button.click() for button in buttons if button.text == "Voir plus"]
sleep(random.uniform(0.2, 0.5))

details = fb.driver.find_elements_by_xpath("//span[@dir]")
for i in range(len(details)):
    detail = details[i].text
    if not db.get("published", False) and KEYWORDS["published"] in detail:
        if KEYWORDS['day'] in detail: # if published less than a week
            db["published"] = int(detail[20])
        else:
            db["published"] = 8
    if not db.get("price", False) and KEYWORDS["price"] in detail:
        db["price"] = int(detail[:5].replace(" ", ""))
    elif not db.get("address", False) and KEYWORDS["address"] in detail:
        db["address"] = detail.replace("Montréal, QC", "").replace(', ', '')
    elif not db.get("surface", False) and KEYWORDS["surface(m2)"] in detail:
        surface = int(detail[:4].replace(" ", ""))
        if surface > 200: # if the seller indicated square meters but is actually square feet
            surface = round(surface / 10.764)
        db["surface"] = surface
    elif not db.get("surface", False) and KEYWORDS["surface(ft2)"] in detail:
        db["surface"] = round(int(detail[:4].replace(" ", "")) / 10.764)
    elif not db.get("bedrooms", False) and KEYWORDS["bedrooms"] in detail:
        db["bedrooms"] = int(detail[0])
    elif not db.get("furnished", False) and detail in KEYWORDS["furnished"]:
        db["furnished"] = detail
    elif not db.get("description", False) and detail == KEYWORDS["description"]:
        db["description"] = details[i + 1].text.replace("Voir moins", "")

db["image"] = set()
last_db_len = -1
while len(db["image"]) != last_db_len:
    last_db_len = len(db["image"])
    img = fb.driver.find_element_by_xpath(
        "//img[@referrerpolicy='origin-when-cross-origin']"
    )
    db["image"].add(img.get_attribute("src"))
    next_button = fb.driver.find_element_by_xpath(
        "//div[@aria-label='Voir l’image suivante']"
    )
    next_button.click()
    sleep(random.uniform(0.6, 1))

for key, value in db.items():
    if key != "image":
        print(key + ":", value)
    # else:
    #     for img_url in db['image']:
    #         response = requests.get(img_url)
    #         display(Image.open(BytesIO(response.content)).resize((400, 400)))

url: https://www.facebook.com/marketplace/item/1296614274099443/
price: 1350
address: 
published: 3
bedrooms: 3
description: Grand 5 1/2 à louer à Montréal St/Leonard.           
3 chambres fermées garde-robe, 1 salle de bain avec entrée laveuse/sécheuse, cuisine fermée avec entrée cuisinière et frigo. 
Proche des commodités, commerces et écoles. 
Accès à l’autoroute 40 et arrêts de bus 141 et 372 à 9 min à pieds.
Idéal pour collocation ou famille. 1350$ charges non inclus. Stationnement disponible payant. 
Disponible maintenant ! 
Texto seulement ou WhatsApp au [hidden information] Kanku 
